In [4]:
!pip install deepl

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import re
import csv
import deepl

In [8]:
# 1) Initialize DeepL client with your API key
#    Replace 'YOUR_DEEPL_API_KEY' with the key you got from DeepL
translator = deepl.Translator("bae6647f-3729-45ee-b970-f1b8a86ebaf4:fx")

def safe_translate(text, target_lang="TR"):
    """
    Translate `text` to Turkish via DeepL.
    Falls back to original if something goes wrong.
    """
    try:
        result = translator.translate_text(text, target_lang=target_lang)
        return result.text
    except Exception:
        return text

def translate_text(text):
    """Translate plain text (outside code blocks)."""
    return safe_translate(text, target_lang="TR")

def translate_comment(line):
    """Translate only the comment part of a code line."""
    m = re.match(r'(\s*(#|//|/\*|\*)\s?)(.*)', line)
    if m:
        prefix, comment = m.group(1), m.group(3)
        if not comment.strip():
            return prefix
        return f"{prefix}{safe_translate(comment, target_lang='TR')}"
    return line

def process_cell(cell):
    """
    Process one question/answer:
    - Split into code vs. non-code
    - Translate non-code wholesale
    - In code, translate only comment lines
    """
    segments = re.split(r'(```[\s\S]*?```)', cell)
    out = []
    for seg in segments:
        if seg.startswith('```') and seg.endswith('```'):
            code_body = seg[3:-3]
            new_lines = [translate_comment(l) for l in code_body.split('\n')]
            out.append('```' + '\n'.join(new_lines) + '```')
        else:
            out.append(translate_text(seg))
    return ''.join(out)

# --- CSV I/O (same as before) ---
in_csv  = './stackoverflow_c_questions_5k.csv'
out_csv = 'translated_dataset_5k.csv'

with open(in_csv,  newline='', encoding='utf-8') as fin, \
     open(out_csv, 'w', newline='', encoding='utf-8') as fout:
    reader = csv.DictReader(fin)
    writer = csv.DictWriter(fout, fieldnames=reader.fieldnames)
    writer.writeheader()

    count = 0
    for row in reader:
        count += 1
        row['question'] = process_cell(row['question'])
        row['answer']   = process_cell(row['answer'])
        writer.writerow(row)

        # Her 100 çeviride bir bildirim ver
        if count % 100 == 0:
            print(f"{count} çeviri tamamlandı...")

print(f"✅ DeepL ile çeviri tamamlandı. Çıktı CSV: {out_csv}")


100 çeviri tamamlandı...
200 çeviri tamamlandı...
300 çeviri tamamlandı...
400 çeviri tamamlandı...
500 çeviri tamamlandı...
600 çeviri tamamlandı...
700 çeviri tamamlandı...


KeyboardInterrupt: 